In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime      # datetime.now() 를 이용하여 학습 경과 시간 측정

# read_data_sets() 를 통해 데이터를 객체형태로 받아오고
# one_hot 옵션을 통해 정답(label) 을 one-hot 인코딩된 형태로 받아옴

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# mnist 데이터 셋은 train, test, validation 3개의 데이터 셋으로 구성되어 있으며.
# num_examples 값을 통해 데이터의 갯수 확인 가능함

print("\n", mnist.train.num_examples, mnist.test.num_examples, mnist.validation.num_examples)

# 데이터는 784(28x28)개의 픽셀을 가지는 이미지와
# 10(0~9)개 클래스를 가지는 one-hot 인코딩된 레이블(정답)을 가지고 있음

print("\ntrain image shape = ", np.shape(mnist.train.images))
print("train label shape = ", np.shape(mnist.train.labels))
print("test image shape = ", np.shape(mnist.test.images))
print("test label shape = ", np.shape(mnist.test.labels))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

 55000 10000 5000

train image shape =  (55000, 784)
train label shape =  (55000, 10)
test image shape =  (10000, 784)
test label shape =  (10000, 10)


In [2]:
# Hyper-Parameter
learning_rate = 0.001  # 학습율
epochs = 30            # 반복횟수
batch_size = 100      # 한번에 입력으로 주어지는 MNIST 개수

In [3]:
# 입력과 정답을 위한 플레이스홀더 정의
X = tf.placeholder(tf.float32, [None, 784])  

A1 = X_img = tf.reshape(X, [-1, 28, 28, 1])   # image 28X28X1 (black/white)


T = tf.placeholder(tf.float32, [None, 10])  

In [4]:
# 1번째 컨볼루션 층, 5X5X32 필터
W2 = tf.Variable(tf.random_normal([5, 5, 1, 32], stddev=0.01))  
#W2 = tf.Variable(tf.random_normal([5, 5, 1, 32]))  
#b2 = tf.Variable(tf.constant(0.1, shape=[32]))  
b2 = tf.Variable(tf.random_normal([32])) 

# 1번째 컨볼루션 연산을 통해 28 X 28 X1  => 28 X 28 X 32 
C2 = tf.nn.conv2d(A1, W2, strides=[1, 1, 1, 1], padding='SAME')

# relu
Z2 = tf.nn.relu(C2+b2)

# 1번째 max pooling을 통해 28 X 28 X 32  => 14 X 14 X 32 
A2 = P2 = tf.nn.max_pool(Z2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
# 14X14 크기를 가진 32개의 activation map을 flatten 시킴
A2_flat = P2_flat = tf.reshape(A2, [-1, 14*14*32])

In [6]:
# 출력층
W3 = tf.Variable(tf.random_normal([14*14*32, 10], stddev=0.01))
#W3 = tf.Variable(tf.random_normal([14*14*32, 10]))
b3 = tf.Variable(tf.random_normal([10]))

# 출력층 선형회귀  값 Z3, 즉 softmax 에 들어가는 입력 값
Z3 = logits = tf.matmul(A2_flat, W3) + b3    # 선형회귀 값 Z3

y = A3 = tf.nn.softmax(Z3)

In [7]:
loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=T) )

optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)  

In [8]:
# batch_size X 10 데이터에 대해 argmax를 통해 행단위로 비교함
predicted_val = tf.equal( tf.argmax(A3, 1), tf.argmax(T, 1) )

# batch_size X 10 의 True, False 를 1 또는 0 으로 변환
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype=tf.float32))

# index list 출력
accuracy_index = tf.cast(predicted_val, dtype=tf.float32)

# 예측값 처리
predicted_list = tf.argmax(A3, 1)

In [9]:
index_label_prediction_list = []

with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화
    
    start_time = datetime.now()
    
    for i in range(epochs):    # 50 번 반복수행
        
        total_batch = int(mnist.train.num_examples / batch_size)  # 55,000 / 100

        for step in range(total_batch):
            
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
      
            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})    
        
            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)             
    
    end_time = datetime.now() 
    
    print("\nelapsed time = ", end_time - start_time) 
    
    # Accuracy 확인
    test_x_data = mnist.test.images    # 10000 X 784
    test_t_data = mnist.test.labels    # 10000 X 10
    
    accuracy_val, predicted_list_val, index_label = sess.run([accuracy, predicted_list, accuracy_index], feed_dict={X: test_x_data, T: test_t_data})
    
    print("\nAccuracy = ", accuracy_val)
    print("type(accuracy_val) = ", type(accuracy_val), ', type(predicted_list_val) = ', type(predicted_list_val), ', type(index_label) = ', type(index_label))
    print("index_label.shape = ", index_label.shape)
    
    index_label_list = list(index_label)
    print("length of index_label_list = ", len(index_label_list))
    print("false label count = ", index_label_list.count([0]))
        
    
    temp_list = [] 
    
    for index in range(len(index_label)):
        
        if index_label[index] == 0:
            
            temp_list.append(index)
            temp_list.append(np.argmax(test_t_data[index]))  # one-hot encoding 이므로 argmax 로 정답 추출
            temp_list.append(predicted_list_val[index])
            
            index_label_prediction_list.append(temp_list)
            
            temp_list = []
            
    print("\nlength of index_label_false_list", len(index_label_prediction_list))

epochs =  0 , step =  0 , loss_val =  2.7350626
epochs =  0 , step =  100 , loss_val =  0.5090084
epochs =  0 , step =  200 , loss_val =  0.3591749
epochs =  0 , step =  300 , loss_val =  0.40368888
epochs =  0 , step =  400 , loss_val =  0.48307198
epochs =  0 , step =  500 , loss_val =  0.37358978
epochs =  1 , step =  0 , loss_val =  0.21785757
epochs =  1 , step =  100 , loss_val =  0.2645036
epochs =  1 , step =  200 , loss_val =  0.13284065
epochs =  1 , step =  300 , loss_val =  0.24669366
epochs =  1 , step =  400 , loss_val =  0.09970439
epochs =  1 , step =  500 , loss_val =  0.08242004
epochs =  2 , step =  0 , loss_val =  0.095352046
epochs =  2 , step =  100 , loss_val =  0.15006094
epochs =  2 , step =  200 , loss_val =  0.08739092
epochs =  2 , step =  300 , loss_val =  0.06122506
epochs =  2 , step =  400 , loss_val =  0.1186985
epochs =  2 , step =  500 , loss_val =  0.04838783
epochs =  3 , step =  0 , loss_val =  0.047051504
epochs =  3 , step =  100 , loss_val =  0.

epochs =  26 , step =  200 , loss_val =  0.0064310418
epochs =  26 , step =  300 , loss_val =  0.0035630933
epochs =  26 , step =  400 , loss_val =  0.008173967
epochs =  26 , step =  500 , loss_val =  0.0029205822
epochs =  27 , step =  0 , loss_val =  0.03768449
epochs =  27 , step =  100 , loss_val =  0.0064491183
epochs =  27 , step =  200 , loss_val =  0.0035185325
epochs =  27 , step =  300 , loss_val =  0.0042545213
epochs =  27 , step =  400 , loss_val =  0.007411126
epochs =  27 , step =  500 , loss_val =  0.0007229367
epochs =  28 , step =  0 , loss_val =  0.03183593
epochs =  28 , step =  100 , loss_val =  0.0051430343
epochs =  28 , step =  200 , loss_val =  0.012519342
epochs =  28 , step =  300 , loss_val =  0.037997454
epochs =  28 , step =  400 , loss_val =  0.00516192
epochs =  28 , step =  500 , loss_val =  0.01920622
epochs =  29 , step =  0 , loss_val =  0.0048880028
epochs =  29 , step =  100 , loss_val =  0.0027064162
epochs =  29 , step =  200 , loss_val =  0.002

In [10]:
# index_label_prediction_list
print(index_label_prediction_list)

[[18, 3, 5], [33, 4, 0], [211, 5, 3], [247, 4, 6], [259, 6, 0], [340, 5, 3], [366, 6, 0], [445, 6, 0], [449, 3, 5], [543, 8, 7], [582, 8, 2], [659, 2, 1], [740, 4, 9], [844, 8, 7], [882, 9, 7], [938, 3, 5], [947, 8, 9], [1014, 6, 5], [1039, 7, 2], [1112, 4, 6], [1178, 4, 0], [1182, 6, 5], [1202, 8, 6], [1226, 7, 2], [1232, 9, 4], [1242, 4, 9], [1247, 9, 5], [1319, 8, 0], [1326, 7, 2], [1393, 5, 3], [1414, 9, 3], [1530, 8, 7], [1543, 7, 3], [1549, 4, 6], [1553, 9, 3], [1621, 0, 6], [1634, 4, 9], [1709, 9, 5], [1717, 8, 0], [1737, 5, 1], [1754, 7, 2], [1790, 2, 7], [1878, 8, 3], [1899, 8, 7], [1901, 9, 4], [2004, 8, 9], [2035, 5, 3], [2098, 2, 0], [2107, 8, 0], [2118, 6, 0], [2130, 4, 9], [2131, 6, 0], [2135, 6, 1], [2272, 8, 0], [2293, 9, 0], [2329, 0, 2], [2406, 9, 4], [2414, 9, 4], [2462, 2, 0], [2597, 5, 3], [2654, 6, 1], [2758, 8, 5], [2771, 4, 9], [2836, 4, 9], [2896, 8, 0], [2921, 3, 2], [2939, 9, 5], [2952, 3, 5], [2953, 3, 5], [2995, 6, 5], [3030, 6, 0], [3289, 8, 9], [3422, 6, 